In [2]:
import transformers

c:\Users\Antoine\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [77]:
from transformers import AutoTokenizer
from transformers import DistilBertForSequenceClassification
from transformers import TextClassificationPipeline

import torch
import torch.nn as nn 
import torch.nn.functional as F
from datasets import load_dataset
import pandas as pd
import json
import gc


import re
import nltk
import numpy as np
from nltk.corpus import stopwords
nltk.download('stopwords')
stopw = stopwords.words('english')

import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import iplot

from tqdm import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Antoine\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [108]:
dataset=load_dataset("banking77")

Generating test split: 100%|██████████| 3080/3080 [00:00<00:00, 50250.33 examples/s]


In [110]:
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, return_tensors="pt")

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_dataset=dataset.map(tokenize,batched=True,remove_columns=['text'])
tokenizer_dataset=tokenizer_dataset.rename_column("label","labels")



loading configuration file config.json from cache at C:\Users\Antoine/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.22.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at C:\Users\Antoine/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411\vocab.txt
loading file tokenizer.json from cache at C:\Users\Antoine/.cache\huggingface\hub\model

In [111]:
tokenizer_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 10003
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 3080
    })
})

In [113]:
tokenizer_dataset["train"].features["labels"]


ClassLabel(names=['activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'card_linking', 'card_not_working', 'card_payment_fee_charged', 'card_payment_not_recognised', 'card_payment_wrong_exchange_rate', 'card_swallowed', 'cash_withdrawal_charge', 'cash_withdrawal_not_recognised', 'change_pin', 'compromised_card', 'contactless_not_working', 'country_support', 'declined_card_payment', 'declined_cash_withdrawal', 'declined_transfer', 'direct_debit_payment_not_recognised', 'disposable_card_limits', 'edit_personal_details', 'exchange_charge', 'exchange_rate', 'exchange_via_app', 'extra_charge_on_statement', 'failed_transfer', 'fiat_currency_support', 'get_disposable_virtual_card', 'get_physical_card', 'getting_spare_card', 'gett

In [115]:
from transformers import AutoModelForSequenceClassification

# Model id to load the tokenizer

# Prepare model labels - useful for inference
labels = tokenizer_dataset["train"].features["labels"].names
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

# Download the model from huggingface.co/models
model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased', num_labels=num_labels, label2id=label2id, id2label=id2label
)

loading configuration file config.json from cache at C:\Users\Antoine/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "activate_my_card",
    "1": "age_limit",
    "10": "card_acceptance",
    "11": "card_arrival",
    "12": "card_delivery_estimate",
    "13": "card_linking",
    "14": "card_not_working",
    "15": "card_payment_fee_charged",
    "16": "card_payment_not_recognised",
    "17": "card_payment_wrong_exchange_rate",
    "18": "card_swallowed",
    "19": "cash_withdrawal_charge",
    "2": "apple_pay_or_google_pay",
    "20": "cash_withdrawal_not_recognised",
    "21": "change_pin",
    "22": "compromised_card",
    "23": "contactless_not_working",


In [84]:
import evaluate
import numpy as np

# Metric Id
metric = evaluate.load("f1")

# Metric helper method
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

In [83]:
from huggingface_hub import HfFolder
from transformers import Trainer, TrainingArguments

In [120]:
repository_id='out/'


training_args = TrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
		num_train_epochs=3,
		# PyTorch 2.0 specifics 
    # bf16=True, # bfloat16 training 
		# torch_compile=True, # optimizations
    optim="adamw_torch", # improved optimizer 
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=200,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    # push to hub parameters
    report_to="tensorboard",
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),

)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenizer_dataset["train"],
    eval_dataset=tokenizer_dataset["test"],
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices


In [121]:
trainer.train()

***** Running training *****
  Num examples = 10003
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1878
  0%|          | 0/1878 [00:00<?, ?it/s]

: 